In [1]:
import dicom2nifti
import nibabel as nib
import nilearn as nil
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
import os
import glob
from pathlib import Path
import SimpleITK as sitk # para calcular rangos
import numpy as np
import io
from PIL import Image
import random
import sys
import pandas as pd
import math
import pandas as pd
import sklearn
import sklearn.metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [2]:
sys.path.append("/Users/hugom/PET-IA/src")

In [3]:
from datasets import ADNIDataset, FleniMyriamDataset, BaseDataset
from transforms import TransformGridImage, ToLabelOutputEBRAINS
from train_lib import train_model, set_parameter_requires_grad, initialize_model
from util import collectAllData
import torch
import torchvision
from torchvision import transforms, utils, models, datasets

In [4]:
config = {
  "last_3_classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxlast_3classes_0_epoch21.pth",
    "num_classes": 3
  },
  "sev_3_classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxmost_severe_3classes_0_epoch31.pth",
    "num_classes": 3
  },
  "visit_3_classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxvisit_3classes_0_epoch10.pth",
    "num_classes": 3
  },
  "last_2classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxlast_2classes_0_epoch18.pth",
    "num_classes": 2
  },
  "sev_2classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxmost_severe_2classes_0_epoch16.pth",
    "num_classes": 2
  },
  "visit_2classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxvisit_2classes_0_epoch5.pth",
    "num_classes": 2
  }
}


In [5]:
def executeModel(dataloader, weights, num_classes):
    featureExtract = False
    dropoutRate = 0.6
    auxEnabled = True
    usePretrained = False
    model_ft, input_size = initialize_model('inception', num_classes, featureExtract, dropoutRate, auxEnabled, use_pretrained=usePretrained)
    device = torch.device(deviceName if torch.cuda.is_available() else "cpu")
    model_state_dict = torch.load(weights, map_location=device)
    model_ft.load_state_dict(model_state_dict)

    model_ft.to(device)
    
    testY, predY = collectAllData(dataloader, model_ft, device, num_classes)
    
    _, preds = torch.max(torch.from_numpy(predY), 1)
    preds = preds.cpu().numpy()
    
    acc = sklearn.metrics.accuracy_score(testY, preds)
    bacc = sklearn.metrics.balanced_accuracy_score(testY, preds)
    
    print(testY)
    print(preds)
    
    print(f"Accuracy: {acc}")
    print(f"B acc: {bacc}")
    matrix = sklearn.metrics.confusion_matrix(testY, preds)
    
    print("Confusion matrix:")
    print(matrix)
    
    if (num_classes == 2):
        #lr_auc = roc_auc_score(testY, torch.from_numpy(predY[:, 1]), average = None)
        #print(f"AUC ROC: {lr_auc}")
        return
    
    print("------------")
    print("Merging 2 with 0")
    
    predY[:, 0] = predY[:, 0] + predY[:, 2] #sumo MCI con CN
    predY = predY[:, 0:2] # Borro MCI
    
    _, preds = torch.max(torch.from_numpy(predY), 1)
    preds = preds.cpu().numpy()
    
    acc = sklearn.metrics.accuracy_score(testY, preds)
    bacc = sklearn.metrics.balanced_accuracy_score(testY, preds)
    
    print(f"Accuracy merged: {acc}")
    print(f"B acc merged: {bacc}")
    
    print("Confusion matrix merged:")
    matrix = sklearn.metrics.confusion_matrix(testY, preds)
    print(matrix)
    
    #lr_auc = roc_auc_score(testY, torch.from_numpy(predY[:, 1]), average = None)
    #print(f"AUC ROC: {lr_auc}")

In [6]:
csvFile = '/Users/hugom/PET-IA/Sets/ebrains.csv'
imagesFolder = '/Users/hugom/PET-IA/Full-DBs/ebrains/data_nifti_stripped/hc_images_preprocessed'
mean = 4959.78173828125
std = 9861.69518896219
means = [mean, mean, mean]
stds = [std, std, std]
ebrainsTransforms = torchvision.transforms.Compose([
    TransformGridImage(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(means, stds)
])
dataset = BaseDataset('ebrains', csvFile, imagesFolder, studyIDLabel = 'ID', transform = ebrainsTransforms, target_transform = ToLabelOutputEBRAINS(), truthLabel = 'Group')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

In [7]:
for key in config:
    print(f"Ejecutando {key}")
    executeModel(dataloader, **config[key])
    print("#################")

Ejecutando last_3_classes


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048


FileNotFoundError: [Errno 2] No such file or directory: '/Users/hugom/PET-IA/Full-DBs/ebrains/data_nifti_stripped/hc_images_preprocessed/s0042855A-207260-00001-000001/metadata.json'